In [1]:
# H1B Hub data from USCIS

#NAICS codes: https://www.census.gov/naics/?58967?yearbck=2022
#left of string: https://datatofish.com/left-right-mid-pandas/

# Download CSV files from USCIS Hub
# Save to git repo

import pandas as pd


In [2]:
#download .csv files
#concat into single dataframe (df)

year_first = 2009
year_last = 2022

data_year_list = []
for year_temp in range(year_first, year_last +1):
    url_temp = f"https://www.uscis.gov/sites/default/files/document/data/h1b_datahubexport-{year_temp}.csv"
    data_year = pd.read_csv(url_temp)
    data_year = data_year.rename(columns={'Initial Approvals':'Initial Approval',
                                         'Initial Denials':'Initial Denial',
                                         'Continuing Approvals':'Continuing Approval',
                                         'Continuing Denials':'Continuing Denial'})
    data_year_list.append(data_year)
    
df = pd.concat(data_year_list, ignore_index=True)
df

C:\Users\jbrob\AppData\Local\Temp\ipykernel_18716\3183627088.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_year = pd.read_csv(url_temp)


,Fiscal Year,Employer,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial,NAICS,Tax ID,State,City,ZIP
0,2009,SAGINAW VLY ST UNIV,1,0,0,0,61,NaN,NaN,UNIVERSITY CENTER,NaN
1,2009,REGIONAL MED CTR BOARD,0,0,1,0,62,NaN,AL,ANNISTON,36202.0
2,2009,BIBB MED CTR BIBB COUNTY HEALTH CA,1,0,0,0,62,NaN,AL,CENTREVILLE,35042.0
3,2009,EMERGENCY MEDICAL SERVICES GROUP,0,0,1,0,62,NaN,AR,FORT SMITH,72917.0
4,2009,THE NEW FOUNDATION,0,1,0,0,61,NaN,AZ,SCOTTSDALE,85271.0
...,...,...,...,...,...,...,...,...,...,...,...
772325,2022,ZYREONECONSULTING LLC,0,0,1,0,54,172.0,AZ,PHOENIX,85004.0
772326,2022,ZYTUS INC,0,0,1,0,54,96.0,VA,HENRICO,23238.0
772327,2022,ZYWAVE INC,0,0,1,0,54,743.0,WI,MILWAUKEE,53226.0
772328,2022,ZYWIE INC,0,0,2,0,54,9321.0,GA,DULUTH,30097.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772330 entries, 0 to 772329
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Fiscal Year          772330 non-null  int64  
 1   Employer             772154 non-null  object 
 2   Initial Approval     772330 non-null  object 
 3   Initial Denial       772330 non-null  object 
 4   Continuing Approval  772330 non-null  object 
 5   Continuing Denial    772330 non-null  object 
 6   NAICS                772330 non-null  int64  
 7   Tax ID               770084 non-null  float64
 8   State                772158 non-null  object 
 9   City                 772305 non-null  object 
 10  ZIP                  772171 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 64.8+ MB


In [4]:
#Clean df

#remove commas from numeric values
cols = ['Initial Approval', 'Initial Denial', 'Continuing Approval', 'Continuing Denial']
df[cols]=df[cols].replace(',','', regex=True)  

#convert data types
df=df.astype({'Fiscal Year':'int32',
              'Initial Approval':'int32',
              'Initial Denial':'int32',
              'Continuing Approval':'int32',
              'Continuing Denial':'int32',
              'NAICS':'str'})
#              'Employer':'str',
#              'Tax ID':'str',
#              'ZIP':'str'})  
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772330 entries, 0 to 772329
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Fiscal Year          772330 non-null  int32  
 1   Employer             772154 non-null  object 
 2   Initial Approval     772330 non-null  int32  
 3   Initial Denial       772330 non-null  int32  
 4   Continuing Approval  772330 non-null  int32  
 5   Continuing Denial    772330 non-null  int32  
 6   NAICS                772330 non-null  object 
 7   Tax ID               770084 non-null  float64
 8   State                772158 non-null  object 
 9   City                 772305 non-null  object 
 10  ZIP                  772171 non-null  float64
dtypes: float64(2), int32(5), object(4)
memory usage: 50.1+ MB


In [5]:
df.head()

,Fiscal Year,Employer,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial,NAICS,Tax ID,State,City,ZIP
0,2009,SAGINAW VLY ST UNIV,1,0,0,0,61,NaN,NaN,UNIVERSITY CENTER,NaN
1,2009,REGIONAL MED CTR BOARD,0,0,1,0,62,NaN,AL,ANNISTON,36202.0
2,2009,BIBB MED CTR BIBB COUNTY HEALTH CA,1,0,0,0,62,NaN,AL,CENTREVILLE,35042.0
3,2009,EMERGENCY MEDICAL SERVICES GROUP,0,0,1,0,62,NaN,AR,FORT SMITH,72917.0
4,2009,THE NEW FOUNDATION,0,1,0,0,61,NaN,AZ,SCOTTSDALE,85271.0


In [6]:
df.describe()

,Fiscal Year,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial,Tax ID,ZIP
count,772330.000000,772330.000000,772330.000000,772330.000000,772330.000000,770084.000000,772171.000000
mean,2015.199561,1.994788,0.239124,3.749336,0.212971,4923.301466,48226.310308
std,4.031026,35.387713,6.320529,74.860628,9.112509,2904.043269,33688.947407
min,2009.000000,0.000000,0.000000,0.000000,0.000000,0.000000,605.000000
25%,2012.000000,0.000000,0.000000,0.000000,0.000000,2378.000000,13026.000000
50%,2015.000000,0.000000,0.000000,1.000000,0.000000,4903.000000,44805.000000
75%,2019.000000,1.000000,0.000000,1.000000,0.000000,7437.000000,80301.000000
max,2022.000000,9191.000000,3070.000000,26145.000000,3910.000000,9999.000000,99929.000000


In [7]:
df

,Fiscal Year,Employer,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial,NAICS,Tax ID,State,City,ZIP
0,2009,SAGINAW VLY ST UNIV,1,0,0,0,61,NaN,NaN,UNIVERSITY CENTER,NaN
1,2009,REGIONAL MED CTR BOARD,0,0,1,0,62,NaN,AL,ANNISTON,36202.0
2,2009,BIBB MED CTR BIBB COUNTY HEALTH CA,1,0,0,0,62,NaN,AL,CENTREVILLE,35042.0
3,2009,EMERGENCY MEDICAL SERVICES GROUP,0,0,1,0,62,NaN,AR,FORT SMITH,72917.0
4,2009,THE NEW FOUNDATION,0,1,0,0,61,NaN,AZ,SCOTTSDALE,85271.0
...,...,...,...,...,...,...,...,...,...,...,...
772325,2022,ZYREONECONSULTING LLC,0,0,1,0,54,172.0,AZ,PHOENIX,85004.0
772326,2022,ZYTUS INC,0,0,1,0,54,96.0,VA,HENRICO,23238.0
772327,2022,ZYWAVE INC,0,0,1,0,54,743.0,WI,MILWAUKEE,53226.0
772328,2022,ZYWIE INC,0,0,2,0,54,9321.0,GA,DULUTH,30097.0


In [8]:
df.isna().sum()

Fiscal Year               0
Employer                176
Initial Approval          0
Initial Denial            0
Continuing Approval       0
Continuing Denial         0
NAICS                     0
Tax ID                 2246
State                   172
City                     25
ZIP                     159
dtype: int64

In [9]:
#electing not to clean State na's, because only 40 fixable State values, with max Approvals = 14

df[(pd.isna(df['State'])) & pd.notna(df['ZIP'])]#.info()  

,Fiscal Year,Employer,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial,NAICS,Tax ID,State,City,ZIP
1783,2009,IMPERVA INC,1,0,0,0,54,133.0,NaN,REDWOOD SHORES,94065.0
7086,2009,IBM CORP,0,0,1,0,54,7985.0,NaN,RSEARCH TRIANGLE PAR,27709.0
7098,2009,IBM CORPORATION,0,0,1,0,54,1985.0,NaN,RESEARCH TRIANGLE PA,27709.0
10365,2009,PEABODY ENERGY CORPORATION,1,0,0,0,21,4153.0,NaN,ST LOUIS,63101.0
18998,2009,GNHSOFT INC,0,0,1,0,54,410.0,NaN,NORCROSS,30093.0
21225,2009,AIRSHIP VENTURES INC ALEXANDRA HAL,0,1,0,0,56,2598.0,NaN,MOFFETT FIELD,94035.0
24869,2009,COMCAST IP SERVICES LLC,0,0,1,0,99,1866.0,NaN,MT LAUREL,8054.0
32187,2009,LIFE GENERATIONS HEALTHCARE LLC,0,1,0,0,62,8455.0,NaN,SANTA ANA HEIGHTS,92707.0
32425,2009,DOTTA FOODS LLC,1,0,0,0,99,3068.0,NaN,ROLLING HILLS EST,90274.0
37990,2009,COLLEGE OF ST CATHERINE,0,0,1,0,61,5509.0,NaN,ST PAUL,55105.0


In [10]:
#Import NAICS table

url = "https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html"

df_naics = pd.read_html(url, header=0)[1]

#"NAICS code 99 means the industry is unknown. 
# Any petition that had a blank code was assigned as 99 as well." (ref. README.md)
df_naics = df_naics.append({'Sector':'99','Description':'unknown'}, ignore_index=True)

df_naics

C:\Users\jbrob\AppData\Local\Temp\ipykernel_18716\634787332.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_naics = df_naics.append({'Sector':'99','Description':'unknown'}, ignore_index=True)


,Sector,Description
0,11,"Agriculture, Forestry, Fishing and Hunting (no..."
1,21,"Mining, Quarrying, and Oil and Gas Extraction"
2,22,Utilities
3,23,Construction
4,31-33,Manufacturing
5,42,Wholesale Trade
6,44-45,Retail Trade
7,48-49,Transportation and Warehousing
8,51,Information
9,52,Finance and Insurance


In [11]:
df_naics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sector       21 non-null     object
 1   Description  21 non-null     object
dtypes: object(2)
memory usage: 464.0+ bytes


In [12]:
df.NAICS.sort_values().unique()

array(['11', '21', '22', '23', '31', '32', '33', '42', '44', '45', '48',
       '49', '51', '52', '53', '54', '55', '56', '61', '62', '71', '72',
       '81', '92', '99'], dtype=object)

In [13]:
#Continuing Approvals by NAICS code

df[['NAICS','Continuing Approval']].groupby('NAICS').sum()\
.sort_values(by='NAICS', ascending=True)


,Continuing Approval
NAICS,
11,1974
21,9373
22,8150
23,9821
31,14502
32,40236
33,244524
42,26902
44,29329


In [14]:
#Replace df.NAICS values to match NAICS table

dict_naics = {'31':'31-33', '32':'31-33', '33':'31-33'
                , '44':'44-45', '45':'44-45'
                , '48':'48-49', '49':'48-49'}

df.NAICS=df.NAICS.replace(dict_naics)  

In [15]:
df.NAICS.sort_values().unique()

array(['11', '21', '22', '23', '31-33', '42', '44-45', '48-49', '51',
       '52', '53', '54', '55', '56', '61', '62', '71', '72', '81', '92',
       '99'], dtype=object)

In [16]:
#Continuing Approvals by NAICS code

df[['NAICS','Continuing Approval']].groupby('NAICS').sum()\
.sort_values(by='NAICS', ascending=True)


,Continuing Approval
NAICS,
11,1974
21,9373
22,8150
23,9821
31-33,299262
42,26902
44-45,102749
48-49,13665
51,191879


In [17]:
df = pd.merge(df, df_naics, how='left', left_on='NAICS', right_on='Sector', suffixes=(False, False))
df

,Fiscal Year,Employer,Initial Approval,Initial Denial,Continuing Approval,Continuing Denial,NAICS,Tax ID,State,City,ZIP,Sector,Description
0,2009,SAGINAW VLY ST UNIV,1,0,0,0,61,NaN,NaN,UNIVERSITY CENTER,NaN,61,Educational Services
1,2009,REGIONAL MED CTR BOARD,0,0,1,0,62,NaN,AL,ANNISTON,36202.0,62,Health Care and Social Assistance
2,2009,BIBB MED CTR BIBB COUNTY HEALTH CA,1,0,0,0,62,NaN,AL,CENTREVILLE,35042.0,62,Health Care and Social Assistance
3,2009,EMERGENCY MEDICAL SERVICES GROUP,0,0,1,0,62,NaN,AR,FORT SMITH,72917.0,62,Health Care and Social Assistance
4,2009,THE NEW FOUNDATION,0,1,0,0,61,NaN,AZ,SCOTTSDALE,85271.0,61,Educational Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...
772325,2022,ZYREONECONSULTING LLC,0,0,1,0,54,172.0,AZ,PHOENIX,85004.0,54,"Professional, Scientific, and Technical Services"
772326,2022,ZYTUS INC,0,0,1,0,54,96.0,VA,HENRICO,23238.0,54,"Professional, Scientific, and Technical Services"
772327,2022,ZYWAVE INC,0,0,1,0,54,743.0,WI,MILWAUKEE,53226.0,54,"Professional, Scientific, and Technical Services"
772328,2022,ZYWIE INC,0,0,2,0,54,9321.0,GA,DULUTH,30097.0,54,"Professional, Scientific, and Technical Services"


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 772330 entries, 0 to 772329
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Fiscal Year          772330 non-null  int32  
 1   Employer             772154 non-null  object 
 2   Initial Approval     772330 non-null  int32  
 3   Initial Denial       772330 non-null  int32  
 4   Continuing Approval  772330 non-null  int32  
 5   Continuing Denial    772330 non-null  int32  
 6   NAICS                772330 non-null  object 
 7   Tax ID               770084 non-null  float64
 8   State                772158 non-null  object 
 9   City                 772305 non-null  object 
 10  ZIP                  772171 non-null  float64
 11  Sector               772330 non-null  object 
 12  Description          772330 non-null  object 
dtypes: float64(2), int32(5), object(6)
memory usage: 67.8+ MB


In [19]:
df.isna().sum()

Fiscal Year               0
Employer                176
Initial Approval          0
Initial Denial            0
Continuing Approval       0
Continuing Denial         0
NAICS                     0
Tax ID                 2246
State                   172
City                     25
ZIP                     159
Sector                    0
Description               0
dtype: int64

In [20]:
#Export df to .csv

#df.to_csv("h1b_hub.csv", index=False, compression = 'infer')

In [21]:
#Export df to compressed .csv

compression_opts = dict(method='zip',
                        archive_name='h1b_hub.csv')  

df.to_csv('h1b_hub.zip', index=False,
          compression=compression_opts)  